# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported into DataFrame
csv_file = os.path.join("../output_data/city_weather.csv")

city_df = pd.read_csv(csv_file)
city_df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
0,Bismarck,US,1613074880,46.8083,-100.7837,90,54,-7.60,11.50
1,Bredasdorp,ZA,1613074880,-34.5322,20.0403,100,88,71.60,8.05
2,Faanui,PF,1613074880,-16.4833,-151.7500,0,72,81.86,16.26
3,Flinders,AU,1613074880,-34.5833,150.8552,100,88,69.01,1.01
4,Bluff,NZ,1613074848,-46.6000,168.3333,38,78,51.91,10.38


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use lat and lng as locations and humidity as the weight
locations = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

# Add heatmap layter to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)
plt.savefig("../Images/HeatMap.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities to fit weather conditions
ideal_weather_df = city_df[(city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 85)]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Wind Speed"] <= 10]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Cloudiness"] == 0]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Humidity"] <= 70]
ideal_weather_df

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
39,São Filipe,CV,1613074888,14.8961,-24.4956,0,68,72.77,9.08
46,Salalah,OM,1613074889,17.0151,54.0924,0,56,71.60,2.30
83,Saint-Philippe,RE,1613074896,-21.3585,55.7679,0,65,80.60,3.44
169,Saint-Pierre,RE,1613074914,-21.3393,55.4781,0,65,80.60,3.44
200,Arlit,NE,1613074919,18.7369,7.3853,0,15,77.61,8.90
289,Bouna,CI,1613074938,9.2667,-3.0000,0,14,83.35,2.57
365,Chabahar,IR,1613074954,25.2919,60.6430,0,49,70.21,4.56
380,Celestún,MX,1613074957,20.8667,-90.4000,0,58,83.95,8.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = ideal_weather_df
hotel_df

# Add a Hotel Name column 
hotel_df['Hotel Name'] = " "
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
39,São Filipe,CV,1613074888,14.8961,-24.4956,0,68,72.77,9.08,
46,Salalah,OM,1613074889,17.0151,54.0924,0,56,71.60,2.30,
83,Saint-Philippe,RE,1613074896,-21.3585,55.7679,0,65,80.60,3.44,
169,Saint-Pierre,RE,1613074914,-21.3393,55.4781,0,65,80.60,3.44,
200,Arlit,NE,1613074919,18.7369,7.3853,0,15,77.61,8.90,
289,Bouna,CI,1613074938,9.2667,-3.0000,0,14,83.35,2.57,
365,Chabahar,IR,1613074954,25.2919,60.6430,0,49,70.21,4.56,
380,Celestún,MX,1613074957,20.8667,-90.4000,0,58,83.95,8.14,


In [7]:
# Set parameters to search for hotels with 5000 meters
params = {
    "radius": 50000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # Get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    # Make request and print url
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        
    except:
        hotel_df.loc[index, "Hotel Name"] = "Hotel Not Found"
        
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
39,São Filipe,CV,1613074888,14.8961,-24.4956,0,68,72.77,9.08,Hotel Xaguate
46,Salalah,OM,1613074889,17.0151,54.0924,0,56,71.60,2.30,Hilton Salalah Resort
83,Saint-Philippe,RE,1613074896,-21.3585,55.7679,0,65,80.60,3.44,LODGE PALMAE
169,Saint-Pierre,RE,1613074914,-21.3393,55.4781,0,65,80.60,3.44,Iloha Seaview Hotel
200,Arlit,NE,1613074919,18.7369,7.3853,0,15,77.61,8.90,Hôtel Telwa Bungalow
289,Bouna,CI,1613074938,9.2667,-3.0000,0,14,83.35,2.57,GNIME'S HÔTEL
365,Chabahar,IR,1613074954,25.2919,60.6430,0,49,70.21,4.56,Shahanhotel
380,Celestún,MX,1613074957,20.8667,-90.4000,0,58,83.95,8.14,Hotel Xixim


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))